# Cat Boost Algorithm.

In [1]:
pip install catboost

   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB ? eta -:--:--
   ---------------------------------------- 0.0/101.1 MB 262.6 kB/s eta 0:06:26
   ---------------------------------------- 0.0/101.1 MB 217.9 kB/s eta 0:07:44
   ---------------------------------------- 0.1/101.1 MB 272.3 kB/s eta 0:06:12
   ---------------------------------------- 0.1/101.1 MB 374.1 kB/s eta 0:04:31
   ---------------------------------------- 0.1/101.1 MB 473.7 kB/s eta 0:03:34
   ---------------------------------------- 0.2/101.1 MB 562.0 kB/s eta 0:03:00
   ---------------------------------------- 0.2/101.1 MB 602.4 kB/s eta 0:02:48
   ---------------------------------------- 0.3/101.1 MB 630.9 kB/s eta 0:02:40
   ---------------------------------------- 0.3/101.1 MB 676.8 kB/s eta 0:02

'DOSKEY' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
# import libararies
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from catboost import CatBoostClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report


<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 216 from C header, got 232 from PyObject


In [2]:
# data import 
df = sns.load_dataset('titanic')
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


# Data preprocessing

In [6]:
df.isnull().sum().sort_values(ascending=False)

deck           688
embarked         2
embark_town      2
survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
class            0
who              0
adult_male       0
alive            0
alone            0
dtype: int64

In [8]:
#impute missing values using knn imputers in fare, age
from sklearn.impute import KNNImputer
imputer = KNNImputer()
df['fare'] = imputer.fit_transform(df[['fare']])
df['age'] = imputer.fit_transform(df[['age']])

#impute missing values using knn imputers in embarked and embark_town
from sklearn.impute import SimpleImputer
si = SimpleImputer(strategy='most_frequent')
df['embarked'] = si.fit_transform(df[['embarked']])
df['embark_town'] = si.fit_transform(df[['embark_town']])


ValueError: 2

In [9]:
# impute embarked missing values using pandas
df['embarked'] = df['embarked'].fillna(df['embarked'].mode()[0])
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

In [12]:
df.isnull().sum().sort_values(ascending=False)

deck           688
survived         0
pclass           0
sex              0
age              0
sibsp            0
parch            0
fare             0
embarked         0
class            0
who              0
adult_male       0
embark_town      0
alive            0
alone            0
dtype: int64

In [16]:
# drop deck kolumn
df = df.drop('deck', axis=1)
df.isnull().sum().sort_values(ascending=False)

survived       0
pclass         0
sex            0
age            0
sibsp          0
parch          0
fare           0
embarked       0
class          0
who            0
adult_male     0
embark_town    0
alive          0
alone          0
dtype: int64

In [19]:
# Convert each kategory kolumn to kategory
kategorikal_kolumns = df.select_dtypes(include=['object', 'category']).columns
kategorikal_kolumns
# add this as a new kolumn in the dataframe
df[kategorikal_kolumns] = df[kategorikal_kolumns].astype('category')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   survived     891 non-null    int64   
 1   pclass       891 non-null    int64   
 2   sex          891 non-null    category
 3   age          891 non-null    float64 
 4   sibsp        891 non-null    int64   
 5   parch        891 non-null    int64   
 6   fare         891 non-null    float64 
 7   embarked     891 non-null    category
 8   class        891 non-null    category
 9   who          891 non-null    category
 10  adult_male   891 non-null    bool    
 11  embark_town  891 non-null    category
 12  alive        891 non-null    category
 13  alone        891 non-null    bool    
dtypes: bool(2), category(6), float64(2), int64(4)
memory usage: 49.6 KB


In [20]:
df.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,Southampton,no,True


In [23]:
# split data into X and y
X = df.drop('survived', axis=1)
y = df.survived

# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
# konvert kategorikla data types into numerikal
# le = LabelEncoder()
# le.fit_transform()

In [29]:
# run the Cat boost kalsifier
kb = CatBoostClassifier()
kb.fit(X_train, y_train, cat_features=kategorikal_kolumns.to_list());
y_pred = kb.predict(X_test)

#evaluate the model
print('Accuracy_skore: \n', accuracy_score(y_pred, y_test))
print('Confusion matrix: \n', confusion_matrix(y_pred, y_test))
print(f'Klassifikation report: \n, {accuracy_score(y_pred, y_test)}')

Learning rate set to 0.008911
0:	learn: 0.6696662	total: 23.6ms	remaining: 23.6s
1:	learn: 0.6472549	total: 50.7ms	remaining: 25.3s
2:	learn: 0.6253850	total: 79.1ms	remaining: 26.3s
3:	learn: 0.6014598	total: 97.8ms	remaining: 24.4s
4:	learn: 0.5813484	total: 127ms	remaining: 25.2s
5:	learn: 0.5571380	total: 143ms	remaining: 23.7s
6:	learn: 0.5344852	total: 157ms	remaining: 22.2s
7:	learn: 0.5175868	total: 184ms	remaining: 22.8s
8:	learn: 0.4999497	total: 214ms	remaining: 23.6s
9:	learn: 0.4789119	total: 226ms	remaining: 22.4s
10:	learn: 0.4634393	total: 263ms	remaining: 23.7s
11:	learn: 0.4439109	total: 277ms	remaining: 22.8s
12:	learn: 0.4269531	total: 300ms	remaining: 22.8s
13:	learn: 0.4122990	total: 333ms	remaining: 23.5s
14:	learn: 0.3949205	total: 348ms	remaining: 22.8s
15:	learn: 0.3816202	total: 379ms	remaining: 23.3s
16:	learn: 0.3670853	total: 399ms	remaining: 23.1s
17:	learn: 0.3542113	total: 420ms	remaining: 22.9s
18:	learn: 0.3393229	total: 430ms	remaining: 22.2s
19:	lea